In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import gaussian_process
from sklearn import linear_model

X = np.genfromtxt("data/X_train.txt")
Y = np.genfromtxt("data/Y_train.txt")
Xte = np.genfromtxt('data/X_test.txt')

'''
Xtr = X[10000:20000]
Xva = X[20000:30000]
Ytr = Y[10000:20000]
Yva = Y[20000:30000]
'''

Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.80)

In [7]:
from sklearn.ensemble import VotingClassifier

#1) Random Forest Classifier 
rf_clf = RandomForestClassifier(n_estimators=20, max_features=6, bootstrap=True, max_depth=20)

#2) K Nearest Neighbors Classifier
knn_clf = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=1,
           weights='uniform')

#3) AdaBoost Classifier
dt = tree.DecisionTreeClassifier(max_depth=3)
ada_clf = AdaBoostClassifier(base_estimator=dt, n_estimators=50)

vclf = VotingClassifier(estimators=[('rf', rf_clf), ('knn', knn_clf), ('ada', ada_clf)], voting='soft')
vclf.fit(Xtr, Ytr)


VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=6, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimat...dom_state=None, splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None))],
         n_jobs=1, voting='soft', weights=None)

In [8]:
pred_soft = vclf.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print roc_auc

0.956711150863


In [9]:
pred_soft = vclf.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print roc_auc

0.742377710304


In [5]:
# Now output a file with two columns, a row ID and a confidence in class 1:
pred_soft = vclf.predict_proba(Xte)
np.savetxt('voting_classify.txt',
np.vstack( (np.arange(len(pred_soft)) , pred_soft[:,1]) ).T,
'%d, %.2f',header='ID,Prob1',comments='',delimiter=',');